In [19]:
import rospy
import time
import math
import jupyros as jr
import numpy as np
import assignment_2_2022.msg
from assignment_2_2022.msg import PlanningAction
import assignment_2_2022.srv
from assignment_2_2022.srv import _node_B_goal
import actionlib
import actionlib.msg
import matplotlib.pyplot as plt
import tf
import ipywidgets as widgets

from geometry_msgs.msg import Pose, Point
from IPython.display import display
from sensor_msgs.msg import LaserScan
from matplotlib.animation import FuncAnimation
from nav_msgs.msg import Odometry
from tf.transformations import quaternion_matrix
from std_srvs.srv import *

%matplotlib widget

In [20]:
global client, status

rospy.init_node('user_interface')
# Initialises the node

client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)
# Access the /reaching_goal rostopic

client.wait_for_server()
#Allows the action server to startup before continuing

status = client.get_state()
# Gets the current state of the client

goal_x = widgets.IntText(description='x-Coord', disabled=False)
goal_y = widgets.IntText(description='y-Coord', disabled=False)
# Coordinate inputs for the desired goal

reaching_goal = widgets.Button(description="Confirm")
confirmed = widgets.Output()
# Button widget that confirms an inputed goal

canceled_goal = widgets.Button(description="Cancel")
cancelled = widgets.Output()
# Button widget that cancels a currently active goal

layout = widgets.VBox([goal_x, goal_y])
button_layout = widgets.HBox([reaching_goal, canceled_goal])
# VBox and HBox widget layout for the Coord inputs and buttons

In [21]:
global client, status

def send_goal():
    goal = assignment_2_2022.msg.PlanningGoal()
    status=client.get_state()
    if not (status==actionlib.GoalStatus.ACTIVE):
        goal.target_pose.pose.position.x = x_goal
        goal.target_pose.pose.position.y = y_goal
        client.send_goal(goal)
        print("Goal has reached the server!")
    else:
        print("You have a currently active goal!")
# Send the goal to the robot

def cancel_goal():
    status = client.get_state()
    if status == actionlib.GoalStatus.ACTIVE:
        client.cancel_goal()
        print("Cancelled!")
    else:
        print("Please input a goal!")
# Cancels a currently active goal

def confirm_button(reaching_goal):
    global x_goal, y_goal
    x_goal = int(goal_x.value)
    y_goal = int(goal_y.value)
    with confirmed:
        send_goal()
reaching_goal.on_click(confirm_button)
# Definition for a click of the confirmed button

def cancel_button(canceled_goal):
    with cancelled:
        cancel_goal()
canceled_goal.on_click(cancel_button)
# Definition for a click of the cancelled button

print("Please enter integer X, Y coordinate values:")
display(layout, button_layout, confirmed, cancelled)
print("Once you have input your first coordinates, run all below cells...")
# Displays the Coord inputs and Confirm/Cancel Buttons

Please enter integer X, Y coordinate values:


Output()

Output()

Once you have input your first coordinates, run all below cells...


In [22]:
global x_goal, y_goal, client, status

class Visualiser:
    def __init__(self):

        self.fig, (self.ax, self.ax2) = plt.subplots(1,2, figsize=(8,4))
        # Creates a subplot each for the robot Trajectory and status of goals
        
        self.ln, = self.ax.plot([], [], 'mo', markersize='3', label='Trajectory')
        self.ax.plot(x_goal, y_goal, marker='x', color='orange', markersize='8', label='Current goal')
        self.ax.plot(x_goal, y_goal, marker='x', color='green', markersize='8', label='Reached goals')
        self.ax.plot(x_goal, y_goal, marker='x', color='red', markersize='8', label='Cancelled goals')
        # Provides distinct labels for each status as well as a graphing line for the trajectory
        
        self.x_data, self.y_data = [] , []
    
        self.reached = 0
        self.cancelled = 0
        self.reached_check=False
        self.cancelled_check=False
        # Initialises checks for the goal status
    
    def plot_init(self):
        self.ax.set_xlim(-15, 15)
        self.ax.set_ylim(-15, 15)
        self.ax2.set_ylim(0, 10)
        return self.ln
    # Definition that sets axis limits for the graphs
    
    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
        current_px_widget.value=round(msg.pose.pose.position.x, 3)
        current_py_widget.value=round(msg.pose.pose.position.y, 3)
    # Definition for standard callback to find the position of the robot
    
    def update_plot(self, frame):
    # Definition for updating the plots
        self.ln.set_data(self.x_data, self.y_data)

        status=client.get_state()
        
        if status==3 and not(self.reached_check):
            self.ax.plot(x_goal, y_goal, marker='x', color='green', markersize='8', label='Reached')
            self.reached += 1
            self.ax2.bar(['reached', 'cancelled'], [self.reached, self.cancelled], color = 'b')
            self.reached_check=True
        # Check to see if goal has been reached
        
        elif (not(status==3) and self.reached_check):
            self.reached_check=False 
            
        elif status==2 and not(self.cancelled_check):
            self.ax.plot(x_goal, y_goal, marker='x', color='red', markersize='8', label='Cancelled')
            self.cancelled += 1
            self.ax2.bar(['reached', 'cancelled'], [self.reached, self.cancelled], color = 'b')
            self.cancelled_check=True
        # Check to see if the goal has been cancelled
            
        elif (not(status==2) and self.cancelled_check):
            self.cancelled_check=False
            
        elif (not(status==3) and not(status==2)):
            self.ax.plot(x_goal, y_goal, marker='x', color='orange', markersize='8', label='Current')
        # Maintains the current marker until one of the above conditions are met
        
        self.ax2.bar(['reached', 'cancelled'], [self.reached, self.cancelled], color = 'c')
        # Shows the second graph
        return self.ln

In [23]:
current_px_widget=widgets.FloatText(description='Robot x-pos:', disabled=True)
# Widget that shows the current x coord of the robot

current_py_widget=widgets.FloatText(description='Robot y-pos:', disabled=True)
# Widget that shows the current y coord of the robot

min_dist_widget =widgets.FloatText( description='Closest obj:', disabled=True)
output_dist=widgets.Output()
# Widget to display the distance to the closest ibstacle picked up by the scanner

In [24]:
vis = Visualiser()
# Call visualiser class

vis.ax.set_title('Plot of robot trajectory')
vis.ax.set_xlabel('Position on x')
vis.ax.set_ylabel('Position on y')
vis.ax.legend()
plt.tight_layout()
vis.ax.grid()
# Trajectory graph

vis.ax2.set_title('Reached and cancelled goals')
vis.ax2.set_ylabel('Number of goals')
# Goal graph

sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
time.sleep(0.2)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
# Standard callback that updates the graphs

widg = widgets.HBox([current_px_widget, current_py_widget, min_dist_widget])
display(widg, output_dist)
# Displays the robots current x and y position as well as distance from the closest object, similar to Node C

plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

In [25]:
global min_range

def scan_callback(scan):
    min_range = scan.range_max
    
    for range_val in scan.ranges:
        if range_val < min_range:
            min_range = range_val
            
    min_dist_widget.value = round(min_range, 3)

jr.subscribe('/scan', LaserScan, scan_callback)
time.sleep(0.3)
# Communicates with the scanner for the distance to object widget

Removing previous callback, only one redirection possible right now
